In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import re
import tkinter as tk
from tkinter import ttk, filedialog
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Define global threading and data storage variables
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Handle pellet events
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {'usb-0:1.1': 'Port 1', 'usb-0:1.2': 'Port 2', 'usb-0:1.3': 'Port 3', 'usb-0:1.4': 'Port 4'}
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if usb_port_path in usb_port_mapping:
                device_mappings.append({'serial_port': serial_port, 'port_identifier': usb_port_mapping[usb_port_path]})
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

def read_from_fed(serial_port, port_identifier, gpio_pins, q):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        q.put("Ready")
        while not stop_event.is_set():
            line = ser.readline().decode('utf-8').strip()
            if line:
                data_list = line.split(",")
                q.put(f"{port_identifier} raw data: {data_list}")
                if len(data_list) >= 10:
                    event_type = data_list[9]
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    data_list[0] = timestamp
                    process_event(event_type, port_identifier, gpio_pins, q)
                    q.put(data_list)
    except serial.SerialException:
        q.put(f"Error opening serial port: {serial_port}")
    finally:
        ser.close()

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.root.configure(bg="black")
        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Cascadia Code", 28, "bold"), bg="black", fg="lavender")
        self.label.pack(expand=True)
        self.root.after(duration, self.close_splash)

    def close_splash(self):
        self.root.destroy()

class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED TTL Monitor")
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}
        self.threads = []  # Initialize threads list

        # Mainframe layout
        self.mainframe = ttk.Frame(self.root)
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Layout Setup
        self.create_layout()
        self.check_connected_devices()
        self.root.after(100, self.update_gui)

    def create_layout(self):
        # Left side: Port 1 and Port 2
        left_ports = ttk.Frame(self.mainframe)
        left_ports.grid(column=0, row=0, sticky=(tk.N, tk.S, tk.W))

        self.setup_port(left_ports, 'Port 1', 0)
        self.setup_port(left_ports, 'Port 2', 1)

        # Right side: Port 3 and Port 4
        right_ports = ttk.Frame(self.mainframe)
        right_ports.grid(column=2, row=0, sticky=(tk.N, tk.S, tk.E))

        self.setup_port(right_ports, 'Port 3', 0)
        self.setup_port(right_ports, 'Port 4', 1)

        # Center controls
        controls_frame = ttk.Frame(self.mainframe)
        controls_frame.grid(column=1, row=0, padx=10, sticky=(tk.N, tk.S))

        tk.Label(controls_frame, text="Your Name:", font=("Cascadia Code", 8)).grid(column=0, row=0, sticky=tk.W)
        self.experimenter_entry = ttk.Entry(controls_frame, textvariable=self.experimenter_name, width=20)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W)

        tk.Label(controls_frame, text="Experiment Name:", font=("Cascadia Code", 8)).grid(column=0, row=1, sticky=tk.W)
        self.experiment_entry = ttk.Entry(controls_frame, textvariable=self.experiment_name, width=20)
        self.experiment_entry.grid(column=1, row=1, sticky=tk.W)

        # Start and Stop buttons
        self.start_button = tk.Button(controls_frame, text="START", font=("Cascadia Code", 9, "bold"), bg="green", fg="white", command=self.start_experiment)
        self.start_button.grid(column=0, row=2, columnspan=2, sticky="we", pady=5)

        self.stop_button = tk.Button(controls_frame, text="STOP & SAVE", font=("Cascadia Code", 9, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=0, row=3, columnspan=2, sticky="we", pady=5)

        # Browse button
        self.browse_button = tk.Button(controls_frame, text="Browse Data Folder", font=("Cascadia Code", 9), command=self.browse_folder)
        self.browse_button.grid(column=0, row=4, columnspan=2, sticky="we", pady=5)

        # Recording Indicator
        self.canvas = tk.Canvas(controls_frame, width=120, height=100)
        self.canvas.grid(column=0, row=5, columnspan=2, pady=8)
        self.recording_circle = None  # Initialize as None
        self.recording_label = None   # Initialize as None

        # Footer copyright message
        self.mainframe.grid_rowconfigure(1, weight=1)
        copyright_label = tk.Label(self.mainframe, text="© 2024 McCutcheonlab | UiT | Norway", font=("Cascadia Code", 8), fg="black")
        copyright_label.grid(column=0, row=2, columnspan=3, sticky="s", pady=5)

    def setup_port(self, parent, port_name, row):
        frame = ttk.LabelFrame(parent, text=port_name, padding="3")
        frame.grid(column=0, row=row, padx=5, pady=5, sticky=(tk.N, tk.S, tk.W, tk.E))
        status_label = ttk.Label(frame, text="Not Ready", font=("Cascadia Code", 10), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)
        text_widget = tk.Text(frame, width=28, height=6, wrap=tk.WORD)
        text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
        self.port_widgets[port_name] = {'status_label': status_label, 'text_widget': text_widget}
        self.port_queues[port_name] = queue.Queue()

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def check_connected_devices(self):
        device_mappings = get_device_mappings_by_usb_port()
        for mapping in device_mappings:
            port_identifier = mapping['port_identifier']
            self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")

    def display_recording_indicator(self):
        if self.recording_circle is None:
            self.recording_circle = self.canvas.create_oval(10, 10, 50, 50, fill="red")
        if self.recording_label is None:
            self.recording_label = self.canvas.create_text(39, 60, text="RECORDING", font=("Cascadia Code", 10),anchor= "n")

    def hide_recording_indicator(self):
        if self.recording_circle is not None:
            self.canvas.delete(self.recording_circle)
            self.recording_circle = None
        if self.recording_label is not None:
            self.canvas.delete(self.recording_label)
            self.recording_label = None

    def start_experiment(self):
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        
        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.data_to_save = {port_identifier: [] for port_identifier in self.port_widgets.keys()}
        self.threads = []  # Initialize threads list
        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            q = self.port_queues[port_identifier]
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q))
            t.daemon = True
            t.start()
            self.threads.append(t)  # Store thread in self.threads
        self.display_recording_indicator()

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Ready", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        stop_event.set()
        for t in self.threads:
            t.join()
        GPIO.cleanup()
        self.save_all_data()
        self.hide_recording_indicator()
        self.root.quit()
        self.root.destroy()

    def save_all_data(self):
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_folder = os.path.join(self.save_path, self.experimenter_name.get().strip())
        experiment_folder = os.path.join(experimenter_folder, f"{self.experiment_name.get().strip()}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)
        for port_identifier, data_rows in self.data_to_save.items():
            # Save data in the experiment folder with folder structures
            filename_user = os.path.join(experiment_folder, f"{port_identifier}_{current_time}.csv")
            with open(filename_user, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                writer.writerows(data_rows)

            # Save data directly into save_path without folder structures
            filename_main = os.path.join(self.save_path, f"{current_time}_{port_identifier}.csv")
            with open(filename_main, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                writer.writerows(data_rows)

if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# Update

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import re
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import csv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Define global threading and data storage variables
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Handle pellet events
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {'usb-0:1.1': 'Port 1', 'usb-0:1.2': 'Port 2', 'usb-0:1.3': 'Port 3', 'usb-0:1.4': 'Port 4'}
    by_path_dir = '/dev/serial/by-path/'
    if not os.path.exists(by_path_dir):
        return device_mappings  # Return empty list if directory doesn't exist
    for symlink in os.listdir(by_path_dir):
        symlink_path = os.path.join(by_path_dir, symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if usb_port_path in usb_port_mapping:
                device_mappings.append({'serial_port': serial_port, 'port_identifier': usb_port_mapping[usb_port_path]})
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

def read_from_fed(serial_port, port_identifier, gpio_pins, q, status_label):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        q.put("Ready")
        status_label.config(text="Connected", foreground="green")
        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    q.put(f"{port_identifier} raw data: {data_list}")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                        data_list[0] = timestamp
                        process_event(event_type, port_identifier, gpio_pins, q)
                        q.put(data_list)
            except serial.SerialException:
                q.put(f"Device on {port_identifier} disconnected.")
                status_label.config(text="Disconnected", foreground="red")
                break  # Exit loop if device is disconnected
    except serial.SerialException:
        q.put(f"Error opening serial port: {serial_port}")
        status_label.config(text="Disconnected", foreground="red")
    finally:
        if 'ser' in locals() and ser.is_open:
            ser.close()
        q.put(f"Stopped reading from {port_identifier}")

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.root.configure(bg="black")
        self.label = tk.Label(self.root, text="McCutcheonlab Technologies", font=("Cascadia Code", 28, "bold"), bg="black", fg="lavender")
        self.label.pack(expand=True)
        self.root.after(duration, self.close_splash)

    def close_splash(self):
        self.root.destroy()

class FED3MonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED TTL Monitor")
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}
        self.threads = []
        self.connected_ports = []
        self.serial_ports = {}  # Store serial objects
        self.logging_active = False

        # Mainframe layout
        self.mainframe = ttk.Frame(self.root)
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Layout Setup
        self.create_layout()
        self.check_connected_devices()
        self.root.after(100, self.update_gui)
        self.root.after(5000, self.refresh_device_status)  # Periodically refresh device status
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def create_layout(self):
        # Left side: Port 1 and Port 2
        left_ports = ttk.Frame(self.mainframe)
        left_ports.grid(column=0, row=0, sticky=(tk.N, tk.S, tk.W))

        self.setup_port(left_ports, 'Port 1', 0)
        self.setup_port(left_ports, 'Port 2', 1)

        # Right side: Port 3 and Port 4
        right_ports = ttk.Frame(self.mainframe)
        right_ports.grid(column=2, row=0, sticky=(tk.N, tk.S, tk.E))

        self.setup_port(right_ports, 'Port 3', 0)
        self.setup_port(right_ports, 'Port 4', 1)

        # Center controls
        controls_frame = ttk.Frame(self.mainframe)
        controls_frame.grid(column=1, row=0, padx=10, sticky=(tk.N, tk.S))

        tk.Label(controls_frame, text="Your Name:", font=("Cascadia Code", 8)).grid(column=0, row=0, sticky=tk.W)
        self.experimenter_entry = ttk.Entry(controls_frame, textvariable=self.experimenter_name, width=20)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W)

        tk.Label(controls_frame, text="Experiment Name:", font=("Cascadia Code", 8)).grid(column=0, row=1, sticky=tk.W)
        self.experiment_entry = ttk.Entry(controls_frame, textvariable=self.experiment_name, width=20)
        self.experiment_entry.grid(column=1, row=1, sticky=tk.W)

        # Start and Stop buttons
        self.start_button = tk.Button(controls_frame, text="START", font=("Cascadia Code", 9, "bold"), bg="green", fg="white", command=self.start_experiment)
        self.start_button.grid(column=0, row=2, columnspan=2, sticky="we", pady=5)

        self.stop_button = tk.Button(controls_frame, text="STOP & SAVE", font=("Cascadia Code", 9, "bold"), bg="red", fg="white", command=self.stop_experiment)
        self.stop_button.grid(column=0, row=3, columnspan=2, sticky="we", pady=5)

        # Browse button
        self.browse_button = tk.Button(controls_frame, text="Browse Data Folder", font=("Cascadia Code", 9), command=self.browse_folder)
        self.browse_button.grid(column=0, row=4, columnspan=2, sticky="we", pady=5)

        # Recording Indicator
        self.canvas = tk.Canvas(controls_frame, width=120, height=100)
        self.canvas.grid(column=0, row=5, columnspan=2, pady=8)
        self.recording_circle = None  # Initialize as None
        self.recording_label = None   # Initialize as None

        # Footer copyright message
        self.mainframe.grid_rowconfigure(1, weight=1)
        copyright_label = tk.Label(self.mainframe, text="© 2024 McCutcheonlab | UiT | Norway", font=("Cascadia Code", 8), fg="black")
        copyright_label.grid(column=0, row=2, columnspan=3, sticky="s", pady=5)

    def setup_port(self, parent, port_name, row):
        frame = ttk.LabelFrame(parent, text=port_name, padding="3")
        frame.grid(column=0, row=row, padx=5, pady=5, sticky=(tk.N, tk.S, tk.W, tk.E))
        status_label = ttk.Label(frame, text="Not Connected", font=("Cascadia Code", 10), foreground="red")
        status_label.grid(column=0, row=0, sticky=tk.W)
        text_widget = tk.Text(frame, width=28, height=6, wrap=tk.WORD)
        text_widget.grid(column=0, row=1, sticky=(tk.N, tk.S, tk.E, tk.W))
        self.port_widgets[port_name] = {'status_label': status_label, 'text_widget': text_widget}
        self.port_queues[port_name] = queue.Queue()

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def check_connected_devices(self):
        device_mappings = get_device_mappings_by_usb_port()
        self.connected_ports = []
        for mapping in device_mappings:
            port_identifier = mapping['port_identifier']
            self.connected_ports.append(port_identifier)
            self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
        # Check for disconnected ports
        for port_name in self.port_widgets.keys():
            if port_name not in self.connected_ports:
                self.port_widgets[port_name]['status_label'].config(text="Not Connected", foreground="red")
        if not self.connected_ports and not self.logging_active:
            messagebox.showwarning("No Devices", "No FED3 devices are connected. Connect your devices.")

    def refresh_device_status(self):
        self.check_connected_devices()
        # Schedule the next check
        self.root.after(5000, self.refresh_device_status)

    def display_recording_indicator(self):
        if self.recording_circle is None:
            self.recording_circle = self.canvas.create_oval(10, 10, 50, 50, fill="red")
        if self.recording_label is None:
            self.recording_label = self.canvas.create_text(30, 60, text="RECORDING", font=("Cascadia Code", 10), anchor="n")

    def hide_recording_indicator(self):
        if self.recording_circle is not None:
            self.canvas.delete(self.recording_circle)
            self.recording_circle = None
        if self.recording_label is not None:
            self.canvas.delete(self.recording_label)
            self.recording_label = None

    def start_experiment(self):
        if not self.connected_ports:
            messagebox.showwarning("No Devices", "No FED3 devices are connected.")
            return
        self.logging_active = True
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.start_button.config(state='disabled')

        self.experimenter_name.set(self.experimenter_name.get().strip().lower())
        self.experiment_name.set(self.experiment_name.get().strip().lower())
        self.data_to_save = {port_identifier: [] for port_identifier in self.port_widgets.keys()}
        self.threads = []

        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            q = self.port_queues[port_identifier]
            status_label = self.port_widgets[port_identifier]['status_label']
            t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label))
            t.daemon = True
            t.start()
            self.threads.append(t)
            self.serial_ports[port_identifier] = serial_port  # Store the serial port

        self.display_recording_indicator()

    def update_gui(self):
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass
        self.root.after(100, self.update_gui)

    def stop_experiment(self):
        if not self.logging_active:
            self.root.quit()
            self.root.destroy()
            return

        stop_event.set()
        for t in self.threads:
            t.join()
        GPIO.cleanup()
        self.save_all_data()
        self.hide_recording_indicator()
        self.logging_active = False
        self.root.quit()
        self.root.destroy()

    def save_all_data(self):
        if not self.save_path:
            messagebox.showerror("Error", "Please select a folder to save data.")
            return
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_name = re.sub(r'[<>:"/\\|?*]', '_', self.experimenter_name.get())
        experiment_name = re.sub(r'[<>:"/\\|?*]', '_', self.experiment_name.get())

        experimenter_folder = os.path.join(self.save_path, experimenter_name)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment_name}_{current_time}")
        os.makedirs(experiment_folder, exist_ok=True)
        for port_identifier, data_rows in self.data_to_save.items():
            if data_rows:
                filename_user = os.path.join(experiment_folder, f"{port_identifier}_{current_time}.csv")
                try:
                    with open(filename_user, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    logging.info(f"Data saved for {port_identifier} in {filename_user}")
                except Exception as e:
                    logging.error(f"Failed to save data for {port_identifier}: {e}")
            else:
                logging.info(f"No data collected from {port_identifier}, no file saved.")
        messagebox.showinfo("Data Saved", "All data has been saved locally.")

    def on_closing(self):
        if self.logging_active:
            if messagebox.askokcancel("Quit", "Logging is active. Do you want to stop and exit?"):
                self.stop_experiment()
        else:
            self.root.quit()
            self.root.destroy()

if __name__ == "__main__":
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root)
    splash_root.mainloop()

    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


# code above works fine, here update dynamics

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import re
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import csv
import gspread
from google.oauth2.service_account import Credentials
import cv2
import webbrowser

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for each device
gpio_pins_per_device = {
    'Port 1': {"LeftPoke": 17, "RightPoke": 27, "Pellet": 22},
    'Port 2': {"LeftPoke": 10, "RightPoke": 9, "Pellet": 11},
    'Port 3': {"LeftPoke": 0, "RightPoke": 5, "Pellet": 6},
    'Port 4': {"LeftPoke": 13, "RightPoke": 19, "Pellet": 26},
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Define global threading and data storage variables
pellet_lock = threading.Lock()
pellet_in_well = {}
stop_event = threading.Event()
column_headers = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets Scope
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

def send_ttl_signal(pin):
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.1)
    GPIO.output(pin, GPIO.LOW)

# Handle pellet events
def handle_pellet_event(event_type, port_identifier, gpio_pins, q):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)
                q.put(f"Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False
                send_ttl_signal(gpio_pins["Pellet"])
            else:
                q.put(f"No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)
            pellet_in_well[port_identifier] = True
            q.put(f"Pellet dispensed in well, signal ON.")

def process_event(event_type, port_identifier, gpio_pins, q):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    message = f"[{timestamp}] {port_identifier} - Event: {event_type}"
    q.put(message)
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftPoke"])
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightPoke"])
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins, q)

def get_device_mappings_by_usb_port():
    device_mappings = []
    usb_port_mapping = {'usb-0:1.1': 'Port 1', 'usb-0:1.2': 'Port 2', 'usb-0:1.3': 'Port 3', 'usb-0:1.4': 'Port 4'}
    by_path_dir = '/dev/serial/by-path/'
    if not os.path.exists(by_path_dir):
        return device_mappings  # Return empty list if directory doesn't exist
    for symlink in os.listdir(by_path_dir):
        symlink_path = os.path.join(by_path_dir, symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if usb_port_path in usb_port_mapping:
                device_mappings.append({'serial_port': serial_port, 'port_identifier': usb_port_mapping[usb_port_path]})
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    return match.group() if match else None

def read_from_fed(serial_port, port_identifier, gpio_pins, q, status_label, google_sheet):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        q.put("Ready")
        status_label.config(text="Connected", foreground="green")
        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    q.put(f"{port_identifier} raw data: {data_list}")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                        data_list[0] = timestamp
                        process_event(event_type, port_identifier, gpio_pins, q)
                        # Prepare data for Google Sheets
                        if len(data_list) == len(column_headers):
                            try:
                                google_sheet.append_row(data_list)
                                q.put(f"Appended data to Google Sheets for {port_identifier}.")
                            except Exception as e:
                                q.put(f"Failed to append data to Google Sheets for {port_identifier}: {e}")
            except serial.SerialException:
                q.put(f"Device on {port_identifier} disconnected.")
                status_label.config(text="Disconnected", foreground="red")
                break  # Exit loop if device is disconnected
    except serial.SerialException:
        q.put(f"Error opening serial port: {serial_port}")
        status_label.config(text="Disconnected", foreground="red")
    finally:
        if 'ser' in locals() and ser.is_open:
            ser.close()
        q.put(f"Stopped reading from {port_identifier}")

class SplashScreen:
    def __init__(self, root, duration=3000):
        self.root = root
        self.root.overrideredirect(True)
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.root.configure(bg="black")
        self.label = tk.Label(
            self.root,
            text="McCutcheonLab Technologies",
            font=("Cascadia Code", 28, "bold"),
            bg="black",
            fg="lavender"
        )
        self.label.pack(expand=True)
        self.root.after(duration, self.close_splash)

    def close_splash(self):
        self.root.destroy()

class FED3MonitorAppGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("HPFED TTL Monitor")
        self.root.geometry("800x480")
        self.root.attributes("-fullscreen", True)

        self.port_widgets = {}
        self.port_queues = {}
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}
        self.threads = []
        self.connected_ports = []
        self.serial_ports = {}  # Store serial objects
        self.logging_active = False

        # Initialize Google Sheets client
        self.gspread_client = None
        self.google_sheets = {}  # Mapping port to Google Sheets worksheet

        # Initialize the last device check time
        self.last_device_check_time = time.time()

        # Retry parameters
        self.retry_attempts = 5
        self.retry_delay = 2  # seconds

        # Setup GUI
        self.setup_gui()

        # Start device detection
        self.check_connected_devices()
        self.root.after(100, self.update_gui)
        self.root.after(5000, self.refresh_device_status)  # Periodically refresh device status

        # Handle window close event
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)

    def setup_gui(self):
        # Mainframe layout
        self.mainframe = ttk.Frame(self.root)
        self.mainframe.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        # Layout Setup
        self.create_layout()

    def create_layout(self):
        # Left side: Port 1 and Port 2
        left_ports = ttk.Frame(self.mainframe)
        left_ports.grid(column=0, row=0, sticky=(tk.N, tk.S, tk.W), padx=5, pady=5)

        self.setup_port(left_ports, 'Port 1', 0)
        self.setup_port(left_ports, 'Port 2', 1)

        # Right side: Port 3 and Port 4
        right_ports = ttk.Frame(self.mainframe)
        right_ports.grid(column=2, row=0, sticky=(tk.N, tk.S, tk.E), padx=5, pady=5)

        self.setup_port(right_ports, 'Port 3', 0)
        self.setup_port(right_ports, 'Port 4', 1)

        # Center controls
        controls_frame = ttk.Frame(self.mainframe)
        controls_frame.grid(column=1, row=0, padx=10, pady=5, sticky=(tk.N, tk.S))

        # Experimenter name input
        tk.Label(
            controls_frame,
            text="Your Name:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=0, row=0, sticky=tk.W, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(
            controls_frame,
            textvariable=self.experimenter_name,
            width=20
        )
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=2)

        # Experiment name input
        tk.Label(
            controls_frame,
            text="Experiment Name:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=0, row=1, sticky=tk.W, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(
            controls_frame,
            textvariable=self.experiment_name,
            width=20
        )
        self.experiment_entry.grid(column=1, row=1, sticky=tk.W, padx=2, pady=2)

        # JSON file path
        tk.Label(
            controls_frame,
            text="Google API JSON File:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=0, row=2, sticky=tk.W, padx=2, pady=2)
        self.json_entry = ttk.Entry(
            controls_frame,
            width=25
        )
        self.json_entry.grid(column=1, row=2, columnspan=2, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(
            controls_frame,
            text="Browse",
            command=self.browse_json
        )
        self.browse_json_button.grid(column=3, row=2, padx=2, pady=2)

        # Google Spreadsheet ID
        tk.Label(
            controls_frame,
            text="Google Spreadsheet ID:",
            font=("Cascadia Code", 8, "bold")
        ).grid(column=0, row=3, sticky=tk.W, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(
            controls_frame,
            width=25
        )
        self.spreadsheet_entry.grid(column=1, row=3, columnspan=2, sticky=tk.W, padx=2, pady=2)

        # Start and Stop buttons
        self.start_button = tk.Button(
            controls_frame,
            text="START",
            font=("Cascadia Code", 10, "bold"),
            bg="green",
            fg="white",
            command=self.start_logging
        )
        self.start_button.grid(column=0, row=4, columnspan=2, sticky="we", pady=5, padx=2)

        self.stop_button = tk.Button(
            controls_frame,
            text="STOP(SAVE & QUIT)",
            font=("Cascadia Code", 10, "bold"),
            bg="red",
            fg="white",
            command=self.stop_logging
        )
        self.stop_button.grid(column=2, row=4, columnspan=2, sticky="we", pady=5, padx=2)

        # Browse button for selecting data folder path
        self.browse_button = tk.Button(
            controls_frame,
            text="Browse Data Folder",
            font=("Cascadia Code", 9),
            command=self.browse_folder,
            bg="gold",
            fg="blue"
        )
        self.browse_button.grid(column=0, row=5, columnspan=4, sticky="we", pady=5, padx=2)

        # Canvas for Recording Indicator
        self.canvas = tk.Canvas(controls_frame, width=120, height=100)
        self.canvas.grid(column=0, row=6, columnspan=4, pady=10)
        self.recording_circle = self.canvas.create_oval(10, 10, 50, 50, fill="red")
        self.recording_label = self.canvas.create_text(30, 60, text="OFF", font=("Cascadia Code", 10), anchor="n")

        # Log Section
        log_frame = ttk.Frame(self.mainframe)
        log_frame.grid(column=0, row=1, columnspan=3, pady=5, sticky=(tk.N, tk.S, tk.E, tk.W))
        self.log_text = tk.Text(log_frame, height=10, width=100, wrap=tk.WORD)
        self.log_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        log_scrollbar = ttk.Scrollbar(
            log_frame,
            orient="vertical",
            command=self.log_text.yview
        )
        log_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.log_text.configure(yscrollcommand=log_scrollbar.set)

        # Bottom Frame for Copyright
        bottom_frame = ttk.Frame(self.mainframe)
        bottom_frame.grid(column=0, row=2, columnspan=3, pady=5, sticky=(tk.S, tk.E, tk.W))

        ttk.Label(
            bottom_frame,
            text="© 2024 McCutcheonLab | UiT | Norway",
            font=("Cascadia Code", 8),
            foreground="royalblue"
        ).pack(pady=2)
        hyperlink_label = tk.Label(
            bottom_frame,
            text="Developed by Hamid Taghipourbibalan",
            font=("Cascadia Code", 8, "italic"),
            fg="blue",
            cursor="hand2"
        )
        hyperlink_label.pack(pady=2)
        hyperlink_label.bind("<Button-1>", lambda e: self.open_hyperlink("https://www.linkedin.com/in/hamid-taghipourbibalan-b7239088/"))

    def open_hyperlink(self, URL):
        webbrowser.open_new(URL)

    def browse_json(self):
        filename = filedialog.askopenfilename(title="Select JSON File", filetypes=[("JSON Files", "*.json")])
        if filename:
            self.json_entry.delete(0, tk.END)
            self.json_entry.insert(0, filename)
            self.log_queue.put(f"Selected JSON file: {filename}")

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")
        if self.save_path:
            self.log_queue.put(f"Data folder selected: {self.save_path}")
            print(f"Save path selected: {self.save_path}")  # For debugging

    def setup_port(self, parent, port_name, row):
        frame = ttk.LabelFrame(parent, text=port_name, padding="5")
        frame.grid(column=0, row=row, padx=5, pady=5, sticky=(tk.N, tk.S, tk.W, tk.E))

        # Status Label
        status_label = ttk.Label(
            frame,
            text="Not Connected",
            font=("Cascadia Code", 10),
            foreground="red"
        )
        status_label.grid(column=0, row=0, sticky=tk.W)

        # Indicator Circle
        indicator_canvas = tk.Canvas(frame, width=20, height=20)
        indicator_canvas.grid(column=1, row=0, padx=5)
        indicator_circle = indicator_canvas.create_oval(5, 5, 15, 15, fill="red")

        # Camera Selection Dropdown
        camera_label = ttk.Label(frame, text="Camera Index:", font=("Cascadia Code", 8))
        camera_label.grid(column=0, row=1, sticky=tk.W, padx=2, pady=2)

        camera_var = tk.StringVar(value='None')
        camera_indices = self.detect_cameras()
        camera_combobox = ttk.Combobox(
            frame,
            textvariable=camera_var,
            values=['None'] + camera_indices,
            width=5,
            state='readonly'
        )
        camera_combobox.grid(column=1, row=1, sticky=tk.W, padx=2, pady=2)

        # Test Camera Button
        test_cam_button = tk.Button(
            frame,
            text="Test Camera",
            command=lambda p=port_name: self.test_camera(p)
        )
        test_cam_button.grid(column=2, row=1, padx=2, pady=2)

        # Text widget for port data
        text_widget = tk.Text(frame, width=35, height=6, wrap=tk.WORD)
        text_widget.grid(column=0, row=2, columnspan=3, sticky=(tk.N, tk.S, tk.E, tk.W))

        # Store widgets in the port_widgets dictionary
        self.port_widgets[port_name] = {
            'status_label': status_label,
            'indicator_canvas': indicator_canvas,
            'indicator_circle': indicator_circle,
            'camera_var': camera_var,
            'camera_combobox': camera_combobox,
            'test_cam_button': test_cam_button,
            'text_widget': text_widget
        }
        self.port_queues[port_name] = queue.Queue()  # Initialize queue for each port

    def test_camera(self, port):
        camera_index_str = self.port_widgets[port]['camera_var'].get()
        if camera_index_str == 'None':
            messagebox.showinfo("Info", "Please select a camera index to test.")
            return
        camera_index = int(camera_index_str)
        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            messagebox.showerror("Error", f"Cannot open camera with index {camera_index}")
            return

        cv2.namedWindow(f"Camera {camera_index} - {port}", cv2.WINDOW_NORMAL)
        cv2.resizeWindow(f"Camera {camera_index} - {port}", 640, 480)
        start_time = time.time()
        while time.time() - start_time < 5:  # Show for 5 seconds
            ret, frame = cap.read()
            if ret:
                cv2.imshow(f"Camera {camera_index} - {port}", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                messagebox.showerror("Error", f"Failed to read from camera {camera_index}")
                break
        cap.release()
        cv2.destroyWindow(f"Camera {camera_index} - {port}")

    def start_logging(self):
        if self.logging_active:
            messagebox.showwarning("Warning", "Logging is already active.")
            return

        # Normalize inputs
        experimenter = self.experimenter_name.get().strip().lower()
        experiment = self.experiment_name.get().strip().lower()
        json_file = self.json_entry.get().strip()
        spreadsheet_id = self.spreadsheet_entry.get().strip()
        save_path = self.save_path.strip()

        # Validate input fields
        if not experimenter or not experiment:
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return

        if not json_file or not spreadsheet_id or not save_path:
            messagebox.showerror("Error", "Please provide the JSON file path, Spreadsheet ID, and select a data folder.")
            return

        # Sanitize names to remove invalid characters
        experimenter = re.sub(r'[<>:"/\\|?*]', '_', experimenter)
        experiment = re.sub(r'[<>:"/\\|?*]', '_', experiment)
        self.experimenter_name.set(experimenter)
        self.experiment_name.set(experiment)

        # Initialize Google Sheets client
        try:
            creds = Credentials.from_service_account_file(json_file, scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to connect to Google Sheets: {e}")
            return

        # Create experiment folder
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_folder = os.path.join(save_path, experimenter)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment}_{current_time}")
        try:
            os.makedirs(experiment_folder, exist_ok=True)
            self.log_queue.put(f"Experiment folder created at {experiment_folder}")
        except Exception as e:
            self.log_queue.put(f"Failed to create directory {experiment_folder}: {e}")
            messagebox.showerror("Error", f"Failed to create directory {experiment_folder}: {e}")
            return

        self.data_to_save = {port: [] for port in self.port_widgets.keys()}

        # Disable input fields after pressing START
        self.disable_input_fields()

        # Change recording indicator to ON
        self.canvas.itemconfig(self.recording_circle, fill="green")
        self.canvas.itemconfig(self.recording_label, text="RECORDING", fill="green")

        # Start logging threads for each connected port
        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if gpio_pins:
                # Initialize Google Sheets worksheet for this port
                try:
                    spreadsheet = self.gspread_client.open_by_key(spreadsheet_id)
                    worksheet = self.get_or_create_worksheet(spreadsheet, f"{port_identifier}_{current_time}")
                    self.google_sheets[port_identifier] = worksheet
                except Exception as e:
                    self.log_queue.put(f"Failed to access Google Sheets for {port_identifier}: {e}")
                    continue

                # Start thread for this port
                q = self.port_queues[port_identifier]
                status_label = self.port_widgets[port_identifier]['status_label']
                t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label, worksheet))
                t.daemon = True
                t.start()
                self.threads.append(t)
                self.serial_ports[port_identifier] = serial_port
                self.log_queue.put(f"Started logging thread for {port_identifier}.")

        self.logging_active = True

    def disable_input_fields(self):
        # Disable the input fields
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.json_entry.config(state='disabled')
        self.spreadsheet_entry.config(state='disabled')
        self.browse_json_button.config(state='disabled')
        self.browse_button.config(state='disabled')
        for widget in self.port_widgets.values():
            widget['camera_combobox'].config(state="disabled")
            widget['test_cam_button'].config(state="disabled")

    def enable_input_fields(self):
        # Enable the input fields
        self.experimenter_entry.config(state='normal')
        self.experiment_entry.config(state='normal')
        self.json_entry.config(state='normal')
        self.spreadsheet_entry.config(state='normal')
        self.browse_json_button.config(state='normal')
        self.browse_button.config(state='normal')
        for widget in self.port_widgets.values():
            widget['camera_combobox'].config(state="readonly")
            widget['test_cam_button'].config(state="normal")

    def get_or_create_worksheet(self, spreadsheet, title):
        try:
            return spreadsheet.worksheet(title)
        except gspread.exceptions.WorksheetNotFound:
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
            return sheet

    def update_gui(self):
        # Update port text widgets and indicators
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
                    elif message == "RIGHT_POKE":
                        self.trigger_indicator(port_identifier)
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass

        # Update log messages
        try:
            while True:
                log_message = self.log_queue.get_nowait()
                timestamp = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S")
                self.log_text.insert(tk.END, f"{timestamp}: {log_message}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass

        # Periodically check device connections every 5 seconds
        current_time = time.time()
        if current_time - self.last_device_check_time >= 5:
            self.check_device_connections()
            self.last_device_check_time = current_time

        # Schedule the next update
        self.root.after(100, self.update_gui)

    def check_device_connections(self):
        current_mappings = get_device_mappings_by_usb_port()
        current_ports = set([mapping['port_identifier'] for mapping in current_mappings])

        # Check for disconnected devices
        for port in list(self.serial_ports.keys()):
            if port not in current_ports:
                # Device disconnected
                self.serial_ports.pop(port, None)
                # Update status label
                if port in self.port_widgets:
                    self.port_widgets[port]['status_label'].config(text="Disconnected", foreground="red")
                    self.port_widgets[port]['indicator_canvas'].itemconfig(self.port_widgets[port]['indicator_circle'], fill="red")
                self.log_queue.put(f"Device on {port} disconnected.")
                # Optionally, stop and clean up threads if necessary

        # Check for newly connected devices
        for mapping in current_mappings:
            port_identifier = mapping['port_identifier']
            if port_identifier not in self.serial_ports and port_identifier not in self.google_sheets:
                # New device connected
                serial_port = mapping['serial_port']
                gpio_pins = gpio_pins_per_device.get(port_identifier)
                if gpio_pins:
                    # Initialize Google Sheets worksheet for this port
                    spreadsheet_id = self.spreadsheet_entry.get().strip()
                    if not spreadsheet_id:
                        self.log_queue.put(f"Spreadsheet ID not provided for {port_identifier}.")
                        continue
                    try:
                        spreadsheet = self.gspread_client.open_by_key(spreadsheet_id)
                        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
                        worksheet = self.get_or_create_worksheet(spreadsheet, f"{port_identifier}_{current_time}")
                        self.google_sheets[port_identifier] = worksheet
                        self.log_queue.put(f"Initialized Google Sheets worksheet for {port_identifier}.")
                    except Exception as e:
                        self.log_queue.put(f"Failed to initialize Google Sheets for {port_identifier}: {e}")
                        continue

                    # Start logging thread for the new port
                    q = self.port_queues[port_identifier]
                    status_label = self.port_widgets[port_identifier]['status_label']
                    t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label, worksheet))
                    t.daemon = True
                    t.start()
                    self.threads.append(t)
                    self.serial_ports[port_identifier] = serial_port
                    self.log_queue.put(f"Started logging thread for {port_identifier}.")

    def trigger_indicator(self, port_identifier):
        indicator_canvas = self.port_widgets[port_identifier]['indicator_canvas']
        indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
        # Start blinking the indicator
        def blink(times):
            if times > 0:
                current_color = indicator_canvas.itemcget(indicator_circle, 'fill')
                next_color = 'yellow' if current_color == 'red' else 'red'
                indicator_canvas.itemconfig(indicator_circle, fill=next_color)
                self.root.after(250, lambda: blink(times -1))
            else:
                # Ensure the indicator is set back to 'red' at the end
                indicator_canvas.itemconfig(indicator_circle, fill='red')
        blink(6)  # Blink 6 times (3 cycles)

    def hide_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="red")
        self.canvas.itemconfig(self.recording_label, text="OFF", fill="red")

    def show_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="green")
        self.canvas.itemconfig(self.recording_label, text="RECORDING", fill="green")

    def start_logging(self):
        if self.logging_active:
            messagebox.showwarning("Warning", "Logging is already active.")
            return

        # Normalize inputs
        experimenter = self.experimenter_name.get().strip().lower()
        experiment = self.experiment_name.get().strip().lower()
        json_file = self.json_entry.get().strip()
        spreadsheet_id = self.spreadsheet_entry.get().strip()
        save_path = self.save_path.strip()

        # Validate input fields
        if not experimenter or not experiment:
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return

        if not json_file or not spreadsheet_id or not save_path:
            messagebox.showerror("Error", "Please provide the JSON file path, Spreadsheet ID, and select a data folder.")
            return

        # Sanitize names to remove invalid characters
        experimenter = re.sub(r'[<>:"/\\|?*]', '_', experimenter)
        experiment = re.sub(r'[<>:"/\\|?*]', '_', experiment)
        self.experimenter_name.set(experimenter)
        self.experiment_name.set(experiment)

        # Initialize Google Sheets client
        try:
            creds = Credentials.from_service_account_file(json_file, scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to connect to Google Sheets: {e}")
            return

        # Create experiment folder
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_folder = os.path.join(save_path, experimenter)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment}_{current_time}")
        try:
            os.makedirs(experiment_folder, exist_ok=True)
            self.log_queue.put(f"Experiment folder created at {experiment_folder}")
        except Exception as e:
            self.log_queue.put(f"Failed to create directory {experiment_folder}: {e}")
            messagebox.showerror("Error", f"Failed to create directory {experiment_folder}: {e}")
            return

        self.data_to_save = {port: [] for port in self.port_widgets.keys()}

        # Disable input fields after pressing START
        self.disable_input_fields()

        # Change recording indicator to ON
        self.show_recording_indicator()

        # Start logging threads for each connected port
        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if gpio_pins:
                # Initialize Google Sheets worksheet for this port
                try:
                    spreadsheet = self.gspread_client.open_by_key(spreadsheet_id)
                    worksheet = self.get_or_create_worksheet(spreadsheet, f"{port_identifier}_{current_time}")
                    self.google_sheets[port_identifier] = worksheet
                except Exception as e:
                    self.log_queue.put(f"Failed to access Google Sheets for {port_identifier}: {e}")
                    continue

                # Start thread for this port
                q = self.port_queues[port_identifier]
                status_label = self.port_widgets[port_identifier]['status_label']
                indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
                t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label, worksheet))
                t.daemon = True
                t.start()
                self.threads.append(t)
                self.serial_ports[port_identifier] = serial_port
                self.log_queue.put(f"Started logging thread for {port_identifier}.")

        self.logging_active = True

    def stop_logging(self):
        if not self.logging_active:
            messagebox.showinfo("Info", "Logging is not active.")
            return

        # Signal threads to stop
        stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging...")

        # Change recording indicator to OFF
        self.hide_recording_indicator()

        # Wait for logging threads to finish
        for t in self.threads:
            t.join(timeout=5)  # Prevent hanging
        self.log_queue.put("All logging threads have been stopped.")

        # Save data
        self.save_all_data()

        # Release webcams
        for cam in self.camera_objects.values():
            cam.release()
        self.log_queue.put("All webcams have been released.")

        # Close serial ports
        for ser in self.serial_ports.values():
            try:
                ser.close()
            except Exception as e:
                self.log_queue.put(f"Failed to close serial port: {e}")
        self.log_queue.put("All serial ports have been closed.")

        # Enable input fields after stopping logging
        self.enable_input_fields()

        # Finalize exit
        self._finalize_exit()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("Error: No save path specified.")
            messagebox.showerror("Error", "No save path specified.")
            return

        experimenter_name = self.experimenter_name.get().strip()
        experiment_name = self.experiment_name.get().strip()
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experiment_folder = os.path.join(self.save_path, experimenter_name, f"{experiment_name}_{current_time}")
        try:
            os.makedirs(experiment_folder, exist_ok=True)
            self.log_queue.put(f"Experiment folder ensured at {experiment_folder}")
        except Exception as e:
            self.log_queue.put(f"Failed to create directory {experiment_folder}: {e}")
            messagebox.showerror("Error", f"Failed to create directory {experiment_folder}: {e}")
            return

        for port, data_rows in self.data_to_save.items():
            if data_rows:  # Only save if there is data
                port_safe = re.sub(r'[<>:"/\\|?*]', '_', port)
                filename = os.path.join(experiment_folder, f"{port_safe}_{current_time}.csv")
                try:
                    with open(filename, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    self.log_queue.put(f"Data saved for {port} in {filename}")
                except Exception as e:
                    self.log_queue.put(f"Failed to save data for {port}: {e}")
                    messagebox.showerror("Error", f"Failed to save data for {port}: {e}")
            else:
                self.log_queue.put(f"No data collected from {port}, no file saved.")

        messagebox.showinfo("Data Saved", "All data has been saved locally.")

    def _finalize_exit(self):
        # Inform the user that data has been saved
        if not self.data_saved:
            messagebox.showinfo("Data Saved", "All data has been saved locally.")
            self.data_saved = True  # Set flag to indicate data has been saved
        # Quit and destroy the GUI
        self.root.quit()
        self.root.destroy()

    def update_gui(self):
        # Update port text widgets and indicators
        for port_identifier, q in self.port_queues.items():
            try:
                while True:
                    message = q.get_nowait()
                    if isinstance(message, list):
                        self.data_to_save[port_identifier].append(message)
                    elif message == "Ready":
                        self.port_widgets[port_identifier]['status_label'].config(text="Connected", foreground="green")
                        self.port_widgets[port_identifier]['indicator_canvas'].itemconfig(self.port_widgets[port_identifier]['indicator_circle'], fill="green")
                    elif message == "RIGHT_POKE":
                        self.trigger_indicator(port_identifier)
                    else:
                        text_widget = self.port_widgets[port_identifier]['text_widget']
                        text_widget.insert(tk.END, message + "\n")
                        text_widget.see(tk.END)
            except queue.Empty:
                pass

        # Update log messages
        try:
            while True:
                log_message = self.log_queue.get_nowait()
                timestamp = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S")
                self.log_text.insert(tk.END, f"{timestamp}: {log_message}\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass

        # Periodically check device connections every 5 seconds
        current_time = time.time()
        if current_time - self.last_device_check_time >= 5:
            self.check_device_connections()
            self.last_device_check_time = current_time

        # Schedule the next update
        self.root.after(100, self.update_gui)

    def check_device_connections(self):
        current_mappings = get_device_mappings_by_usb_port()
        current_ports = set([mapping['port_identifier'] for mapping in current_mappings])

        # Check for disconnected devices
        for port in list(self.serial_ports.keys()):
            if port not in current_ports:
                # Device disconnected
                self.serial_ports.pop(port, None)
                # Update status label
                if port in self.port_widgets:
                    self.port_widgets[port]['status_label'].config(text="Disconnected", foreground="red")
                    self.port_widgets[port]['indicator_canvas'].itemconfig(self.port_widgets[port]['indicator_circle'], fill="red")
                self.log_queue.put(f"Device on {port} disconnected.")
                # Optionally, stop and clean up threads if necessary

        # Check for newly connected devices
        for mapping in current_mappings:
            port_identifier = mapping['port_identifier']
            if port_identifier not in self.serial_ports and port_identifier not in self.google_sheets:
                # New device connected
                serial_port = mapping['serial_port']
                gpio_pins = gpio_pins_per_device.get(port_identifier)
                if gpio_pins:
                    # Initialize Google Sheets worksheet for this port
                    spreadsheet_id = self.spreadsheet_entry.get().strip()
                    if not spreadsheet_id:
                        self.log_queue.put(f"Spreadsheet ID not provided for {port_identifier}.")
                        continue
                    try:
                        spreadsheet = self.gspread_client.open_by_key(spreadsheet_id)
                        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
                        worksheet = self.get_or_create_worksheet(spreadsheet, f"{port_identifier}_{current_time}")
                        self.google_sheets[port_identifier] = worksheet
                        self.log_queue.put(f"Initialized Google Sheets worksheet for {port_identifier}.")
                    except Exception as e:
                        self.log_queue.put(f"Failed to initialize Google Sheets for {port_identifier}: {e}")
                        continue

                    # Start logging thread for the new port
                    q = self.port_queues[port_identifier]
                    status_label = self.port_widgets[port_identifier]['status_label']
                    t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label, worksheet))
                    t.daemon = True
                    t.start()
                    self.threads.append(t)
                    self.serial_ports[port_identifier] = serial_port
                    self.log_queue.put(f"Started logging thread for {port_identifier}.")

    def trigger_indicator(self, port_identifier):
        indicator_canvas = self.port_widgets[port_identifier]['indicator_canvas']
        indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
        # Start blinking the indicator
        def blink(times):
            if times > 0:
                current_color = indicator_canvas.itemcget(indicator_circle, 'fill')
                next_color = 'yellow' if current_color == 'red' else 'red'
                indicator_canvas.itemconfig(indicator_circle, fill=next_color)
                self.root.after(250, lambda: blink(times -1))
            else:
                # Ensure the indicator is set back to 'red' at the end
                indicator_canvas.itemconfig(indicator_circle, fill='red')
        blink(6)  # Blink 6 times (3 cycles)

    def stop_experiment(self):
        # Signal threads to stop
        stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging...")

        # Change recording indicator to OFF
        self.hide_recording_indicator()

        # Wait for logging threads to finish
        for t in self.threads:
            t.join(timeout=5)  # Prevent hanging
        self.log_queue.put("All logging threads have been stopped.")

        # Save data
        self.save_all_data()

        # Release webcams
        for cam in self.camera_objects.values():
            cam.release()
        self.log_queue.put("All webcams have been released.")

        # Close serial ports
        for ser in self.serial_ports.values():
            try:
                ser.close()
            except Exception as e:
                self.log_queue.put(f"Failed to close serial port: {e}")
        self.log_queue.put("All serial ports have been closed.")

        # Enable input fields after stopping logging
        self.enable_input_fields()

        # Finalize exit
        self._finalize_exit()

    def stop_logging(self):
        if not self.logging_active:
            messagebox.showinfo("Info", "Logging is not active.")
            return
        if messagebox.askokcancel("Quit", "Do you want to stop logging and quit?"):
            self.stop_experiment()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("Error: No save path specified.")
            messagebox.showerror("Error", "No save path specified.")
            return

        experimenter_name = self.experimenter_name.get().strip()
        experiment_name = self.experiment_name.get().strip()
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experiment_folder = os.path.join(self.save_path, experimenter_name, f"{experiment_name}_{current_time}")
        try:
            os.makedirs(experiment_folder, exist_ok=True)
            self.log_queue.put(f"Experiment folder ensured at {experiment_folder}")
        except Exception as e:
            self.log_queue.put(f"Failed to create directory {experiment_folder}: {e}")
            messagebox.showerror("Error", f"Failed to create directory {experiment_folder}: {e}")
            return

        for port, data_rows in self.data_to_save.items():
            if data_rows:  # Only save if there is data
                port_safe = re.sub(r'[<>:"/\\|?*]', '_', port)
                filename = os.path.join(experiment_folder, f"{port_safe}_{current_time}.csv")
                try:
                    with open(filename, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    self.log_queue.put(f"Data saved for {port} in {filename}")
                except Exception as e:
                    self.log_queue.put(f"Failed to save data for {port}: {e}")
                    messagebox.showerror("Error", f"Failed to save data for {port}: {e}")
            else:
                self.log_queue.put(f"No data collected from {port}, no file saved.")

    def _finalize_exit(self):
        # Inform the user that data has been saved
        if not self.data_saved:
            messagebox.showinfo("Data Saved", "All data has been saved locally.")
            self.data_saved = True  # Set flag to indicate data has been saved
        # Quit and destroy the GUI
        self.root.quit()
        self.root.destroy()

    def disable_input_fields(self):
        # Disable the input fields
        self.experimenter_entry.config(state='disabled')
        self.experiment_entry.config(state='disabled')
        self.json_entry.config(state='disabled')
        self.spreadsheet_entry.config(state='disabled')
        self.browse_json_button.config(state='disabled')
        self.browse_button.config(state='disabled')
        for widget in self.port_widgets.values():
            widget['camera_combobox'].config(state="disabled")
            widget['test_cam_button'].config(state="disabled")

    def enable_input_fields(self):
        # Enable the input fields
        self.experimenter_entry.config(state='normal')
        self.experiment_entry.config(state='normal')
        self.json_entry.config(state='normal')
        self.spreadsheet_entry.config(state='normal')
        self.browse_json_button.config(state='normal')
        self.browse_button.config(state='normal')
        for widget in self.port_widgets.values():
            widget['camera_combobox'].config(state="readonly")
            widget['test_cam_button'].config(state="normal")

    def trigger_indicator(self, port_identifier):
        indicator_canvas = self.port_widgets[port_identifier]['indicator_canvas']
        indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
        # Start blinking the indicator
        def blink(times):
            if times > 0:
                current_color = indicator_canvas.itemcget(indicator_circle, 'fill')
                next_color = 'yellow' if current_color == 'red' else 'red'
                indicator_canvas.itemconfig(indicator_circle, fill=next_color)
                self.root.after(250, lambda: blink(times -1))
            else:
                # Ensure the indicator is set back to 'red' at the end
                indicator_canvas.itemconfig(indicator_circle, fill='red')
        blink(6)  # Blink 6 times (3 cycles)

    def show_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="green")
        self.canvas.itemconfig(self.recording_label, text="RECORDING", fill="green")

    def hide_recording_indicator(self):
        self.canvas.itemconfig(self.recording_circle, fill="red")
        self.canvas.itemconfig(self.recording_label, text="OFF", fill="red")

    def open_hyperlink(self, URL):
        webbrowser.open_new(URL)

    def test_camera(self, port):
        camera_index_str = self.port_widgets[port]['camera_var'].get()
        if camera_index_str == 'None':
            messagebox.showinfo("Info", "Please select a camera index to test.")
            return
        camera_index = int(camera_index_str)
        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            messagebox.showerror("Error", f"Cannot open camera with index {camera_index}")
            return

        cv2.namedWindow(f"Camera {camera_index} - {port}", cv2.WINDOW_NORMAL)
        cv2.resizeWindow(f"Camera {camera_index} - {port}", 640, 480)
        start_time = time.time()
        while time.time() - start_time < 5:  # Show for 5 seconds
            ret, frame = cap.read()
            if ret:
                cv2.imshow(f"Camera {camera_index} - {port}", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                messagebox.showerror("Error", f"Failed to read from camera {camera_index}")
                break
        cap.release()
        cv2.destroyWindow(f"Camera {camera_index} - {port}")

    def start_logging(self):
        if self.logging_active:
            messagebox.showwarning("Warning", "Logging is already active.")
            return

        # Normalize inputs
        experimenter = self.experimenter_name.get().strip().lower()
        experiment = self.experiment_name.get().strip().lower()
        json_file = self.json_entry.get().strip()
        spreadsheet_id = self.spreadsheet_entry.get().strip()
        save_path = self.save_path.strip()

        # Validate input fields
        if not experimenter or not experiment:
            messagebox.showerror("Error", "Please provide your name and experiment name.")
            return

        if not json_file or not spreadsheet_id or not save_path:
            messagebox.showerror("Error", "Please provide the JSON file path, Spreadsheet ID, and select a data folder.")
            return

        # Sanitize names to remove invalid characters
        experimenter = re.sub(r'[<>:"/\\|?*]', '_', experimenter)
        experiment = re.sub(r'[<>:"/\\|?*]', '_', experiment)
        self.experimenter_name.set(experimenter)
        self.experiment_name.set(experiment)

        # Initialize Google Sheets client
        try:
            creds = Credentials.from_service_account_file(json_file, scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            self.log_queue.put("Connected to Google Sheets!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to connect to Google Sheets: {e}")
            return

        # Create experiment folder
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experimenter_folder = os.path.join(save_path, experimenter)
        experiment_folder = os.path.join(experimenter_folder, f"{experiment}_{current_time}")
        try:
            os.makedirs(experiment_folder, exist_ok=True)
            self.log_queue.put(f"Experiment folder created at {experiment_folder}")
        except Exception as e:
            self.log_queue.put(f"Failed to create directory {experiment_folder}: {e}")
            messagebox.showerror("Error", f"Failed to create directory {experiment_folder}: {e}")
            return

        self.data_to_save = {port: [] for port in self.port_widgets.keys()}

        # Disable input fields after pressing START
        self.disable_input_fields()

        # Change recording indicator to ON
        self.show_recording_indicator()

        # Start logging threads for each connected port
        for mapping in get_device_mappings_by_usb_port():
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            gpio_pins = gpio_pins_per_device.get(port_identifier)
            if gpio_pins:
                # Initialize Google Sheets worksheet for this port
                try:
                    spreadsheet = self.gspread_client.open_by_key(spreadsheet_id)
                    worksheet = self.get_or_create_worksheet(spreadsheet, f"{port_identifier}_{current_time}")
                    self.google_sheets[port_identifier] = worksheet
                except Exception as e:
                    self.log_queue.put(f"Failed to access Google Sheets for {port_identifier}: {e}")
                    continue

                # Start thread for this port
                q = self.port_queues[port_identifier]
                status_label = self.port_widgets[port_identifier]['status_label']
                indicator_circle = self.port_widgets[port_identifier]['indicator_circle']
                t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins, q, status_label, worksheet))
                t.daemon = True
                t.start()
                self.threads.append(t)
                self.serial_ports[port_identifier] = serial_port
                self.log_queue.put(f"Started logging thread for {port_identifier}.")

        self.logging_active = True

    def stop_experiment(self):
        if not self.logging_active:
            self.root.quit()
            self.root.destroy()
            return

        # Signal threads to stop
        stop_event.set()
        self.logging_active = False
        self.log_queue.put("Stopping logging...")

        # Change recording indicator to OFF
        self.hide_recording_indicator()

        # Wait for logging threads to finish
        for t in self.threads:
            t.join(timeout=5)  # Prevent hanging
        self.log_queue.put("All logging threads have been stopped.")

        # Save data
        self.save_all_data()

        # Release webcams
        for cam in self.camera_objects.values():
            cam.release()
        self.log_queue.put("All webcams have been released.")

        # Close serial ports
        for ser in self.serial_ports.values():
            try:
                ser.close()
            except Exception as e:
                self.log_queue.put(f"Failed to close serial port: {e}")
        self.log_queue.put("All serial ports have been closed.")

        # Enable input fields after stopping logging
        self.enable_input_fields()

        # Finalize exit
        self._finalize_exit()

    def stop_logging(self):
        if not self.logging_active:
            messagebox.showinfo("Info", "Logging is not active.")
            return
        if messagebox.askokcancel("Quit", "Do you want to stop logging and quit?"):
            self.stop_experiment()

    def save_all_data(self):
        if not self.save_path:
            self.log_queue.put("Error: No save path specified.")
            messagebox.showerror("Error", "No save path specified.")
            return

        experimenter_name = self.experimenter_name.get().strip()
        experiment_name = self.experiment_name.get().strip()
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        experiment_folder = os.path.join(self.save_path, experimenter_name, f"{experiment_name}_{current_time}")
        try:
            os.makedirs(experiment_folder, exist_ok=True)
            self.log_queue.put(f"Experiment folder ensured at {experiment_folder}")
        except Exception as e:
            self.log_queue.put(f"Failed to create directory {experiment_folder}: {e}")
            messagebox.showerror("Error", f"Failed to create directory {experiment_folder}: {e}")
            return

        for port, data_rows in self.data_to_save.items():
            if data_rows:  # Only save if there is data
                port_safe = re.sub(r'[<>:"/\\|?*]', '_', port)
                filename = os.path.join(experiment_folder, f"{port_safe}_{current_time}.csv")
                try:
                    with open(filename, mode='w', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(column_headers)
                        writer.writerows(data_rows)
                    self.log_queue.put(f"Data saved for {port} in {filename}")
                except Exception as e:
                    self.log_queue.put(f"Failed to save data for {port}: {e}")
                    messagebox.showerror("Error", f"Failed to save data for {port}: {e}")
            else:
                self.log_queue.put(f"No data collected from {port}, no file saved.")

    def _finalize_exit(self):
        # Inform the user that data has been saved
        if not self.data_saved:
            messagebox.showinfo("Data Saved", "All data has been saved locally.")
            self.data_saved = True  # Set flag to indicate data has been saved
        # Quit and destroy the GUI
        self.root.quit()
        self.root.destroy()

    def on_closing(self):
        if self.logging_active:
            if messagebox.askokcancel("Quit", "Logging is active. Do you want to stop and exit?"):
                self.stop_experiment()
        else:
            self.root.quit()
            self.root.destroy()

# Main execution
if __name__ == "__main__":
    # Splash screen
    splash_root = tk.Tk()
    splash_screen = SplashScreen(splash_root, duration=7000)  # 7 seconds duration
    splash_root.after(7000, splash_screen.close)
    splash_root.mainloop()

    # Main GUI
    root = tk.Tk()
    app = FED3MonitorAppGUI(root)
    root.mainloop()

    # Cleanup GPIO on exit
    GPIO.cleanup()
